<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/Image/Soma_Size/Cell_pose_AR_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
!pip install matplotlib --quiet

In [93]:

import tifffile
import os
import numpy as np
import matplotlib.pyplot as plt


In [94]:
source_directory = 'C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/NeunSoma/'


my_tiffs = list()
for rt,dirs,file_list in os.walk(source_directory):
    for file in file_list:
        if 'tif' in file:
            my_tiffs.append(os.path.join(rt,file))
print(my_tiffs)

curr_tif = my_tiffs[0]

['C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/NeunSoma/MDGA2_08-22-22_DG_40X_2XZoom_1.tif', 'C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/NeunSoma/MDGA2_08-22-22_S001_DG_40X_2XZoom_1.tif', 'C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/NeunSoma/MDGA2_08-22-22__S001_DG_40X_2XZoom_1.tif', 'C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/NeunSoma/MDGA2_08-24-22_DG_40X_2XZoom_1.tif', 'C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/NeunSoma/MDGA2_08-24-22__S001_DG_40X_2XZoom_1.tif', 'C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/NeunSoma/MDGA2_08-25-22_DG_40X_2XZoom_1.tif', 'C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/NeunSoma/MDGA2_08-25-22__S001_DG_40X_2XZoom_1.tif']


In [103]:
curr_tif = 'C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_Hyper+Crisper/NeunSoma/MDGA2_08-25-22__S001_DG_40X_2XZoom_1.tif'

In [104]:
tiff_format = 'zcyx'
color_dim = tiff_format.index('c')
cind = list(range(len(tiff_format)))
reord = [cind[color_dim]] + cind[:color_dim] + cind[color_dim+1:]

In [105]:
def dimension_to_front(data,dim_x):
    dim_list = list(np.arange(len(data.shape)))
    transpose = [dim_list[dim_x]] + dim_list[:dim_x] + dim_list[dim_x+1:]
    data_t = data.transpose(transpose)
    invs_transpose = list(np.argsort(transpose))
    return data_t, transpose, invs_transpose

def norm_channels(im_data,tiff_format):
    color_dim = tiff_format.index('c')
    data_t, transpose, invs_transpose = dimension_to_front(im_data,color_dim)
    num_channels = data_t.shape[0]
    for c in np.arange(num_channels):
        c_data = data_t[c,:]
        (top,bot) = np.percentile(c_data.flatten(),[99,1])
        c_data = (c_data-bot)/(top-bot)
        data_t[c,:] = c_data
    im_data = data_t.transpose(invs_transpose)
    return im_data
# dimension_to_front(tiff_data,2)

In [124]:
tiff_format = 'zcyx'
tiff_data = tifffile.imread(curr_tif)
tiff_data = norm_channels(tiff_data,'zcyx')



In [ ]:
for z in range(im_data.shape[0]):
    num_c = im_data.shape[1]
    fig,ax=plt.subplots(1,num_c,figsize=(num_c*2,1.5),dpi=300)
    colors = ['Blues','Reds','Greens','gray_r']
    for c in range(num_c):
        ax[c].imshow(im_data[z,c], cmap=colors[c],vmin=0,vmax=1)
        ax[c].axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
!pip install cellpose
from cellpose import models


In [41]:
print(tiff_data.shape)
cell_channel = 3
nuclear_channel = 0
channel_axis = 1

(15, 3, 800, 800)


In [ ]:
model = models.Cellpose(model_type='cyto')
results = model.eval(tiff_data,diameter=30.0,do_3D=True)




In [ ]:
image_znorm, image_r,chan_dim,z_dim = norm_channels(image,chan_dim,z_dim,)
diameter = 30
exp_dist = 5
masks, exp_masks = mask_image_cellpose(image_r,cell_chan,diameter,exp_dist)

In [ ]:
eval(x, batch_size=8, channels=[0, 0], channel_axis=None, invert=False, normalize=True, diameter=30.0, do_3D=False, **kwargs)